#### Importing Necessary libraries

In [1]:
from IPython import display
import math
from pprint import pprint
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid', context='talk', palette='Dark2')

In [2]:
import praw

reddit = praw.Reddit(client_id='aiWW79orV0Vt-wbDP9n7LA',
                     client_secret='BcAUksyRZPCj4kgZRg_dS8_5XHtIQg',
                     user_agent='Assignment')

### Data Collection:

In [3]:
trials = set()

In [4]:
for submission in reddit.subreddit('clinicaltrials').new(limit=None):
    trials.add(submission.title)
    display.clear_output()
    print(submission.url)
    print(len(trials))

https://www.reddit.com/r/clinicaltrials/comments/bguvdx/clinical_trial_for_healthy_malesfemales_in_san/
47


### Sentiment analysis:

In [5]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

sia = SIA()
results = []

for line in trials:
    pol_score = sia.polarity_scores(line)
    pol_score['trials'] = line
    results.append(pol_score)

pprint(results[:3], width=100)

[{'compound': 0.6124,
  'neg': 0.0,
  'neu': 0.667,
  'pos': 0.333,
  'trials': '158 Recently Updated Clinical Trials - actively recruiting healthy volunteers as of '
            '2021-07-15'},
 {'compound': 0.6124,
  'neg': 0.0,
  'neu': 0.667,
  'pos': 0.333,
  'trials': '100 Recently Updated Clinical Trials - actively recruiting healthy volunteers as of '
            '2021-07-21'},
 {'compound': 0.4995, 'neg': 0.0, 'neu': 0.382, 'pos': 0.618, 'trials': 'Covans Dallas TX'}]


In [6]:
df = pd.DataFrame.from_records(results)
df.head()

,neg,neu,pos,compound,trials
0,0.0,0.667,0.333,0.6124,158 Recently Updated Clinical Trials - activel...
1,0.0,0.667,0.333,0.6124,100 Recently Updated Clinical Trials - activel...
2,0.0,0.382,0.618,0.4995,Covans Dallas TX
3,0.0,1.000,0.000,0.0000,The Story of Mucinex - How a Common Drug Becam...
4,0.0,1.000,0.000,0.0000,CBD RESEARCH STUDY: PTSD AND TBI (NYC)


#### Assign Labels Based on Compound Sentiment Score

In [7]:
df['label'] = 0
df.loc[df['compound'] > 0.2, 'label'] = 1
df.loc[df['compound'] < -0.2, 'label'] = -1
df.head()

,neg,neu,pos,compound,trials,label
0,0.0,0.667,0.333,0.6124,158 Recently Updated Clinical Trials - activel...,1
1,0.0,0.667,0.333,0.6124,100 Recently Updated Clinical Trials - activel...,1
2,0.0,0.382,0.618,0.4995,Covans Dallas TX,1
3,0.0,1.000,0.000,0.0000,The Story of Mucinex - How a Common Drug Becam...,0
4,0.0,1.000,0.000,0.0000,CBD RESEARCH STUDY: PTSD AND TBI (NYC),0


#### Saving Labeled Data to CSV

In [8]:
df2 = df[['trials', 'label']]
df2.to_csv('reddit_trials_labels.csv', mode='a', encoding='utf-8', index=False)

#### Counting Sentiment Labels

In [9]:
df.label.value_counts()

 0    24
 1    19
-1     4
Name: label, dtype: int64

#### Load sentiment analysis data

In [11]:
sentiment_data = pd.read_csv('reddit_trials_labels.csv')

### Message Generation

In [14]:
def generate_message(sentiment):
    if sentiment == 1:
        return "Would you like to participate in other trials. Need similar recommendations?"
    elif sentiment == 0:
        return "Let us know if you have any questions or if we can provide more information so you can be a part of clinical trials."
    else:
        return "Have you participated in clinical trials before? Do have any queries regarding participation in clinical trials?"
        
sentiment_data['message'] = sentiment_data['label'].apply(generate_message)
sentiment_data[['trials', 'message']].head()

,trials,message
0,Video on Eroom's Law (Opposite of Moore's Law ...,Have you participated in clinical trials befor...
1,59 Recently Updated Clinical Trials - actively...,Have you participated in clinical trials befor...
2,Research Study for Co-occurring Chronic Pain a...,Have you participated in clinical trials befor...
3,120 Recently Updated Clinical Trials - activel...,Have you participated in clinical trials befor...
4,Webinar Tomorrow: Hear how experiential data a...,Have you participated in clinical trials befor...
